You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__ are currently saved in `office_addresses.csv`. If the value for office is `NaN`, then the employee is remote. 
- __Employee addresses__ are saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ are saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. However, this sheet was edited at some point, and the headers were removed! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ have been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file (the first key is Employee ID):
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
```

In [59]:
# Start your code here!
import pandas as pd

In [60]:
addresses = pd.read_csv('datasets/office_addresses.csv')

addresses

,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


In [61]:
employees1 = pd.read_excel('datasets/employee_information.xlsx', sheet_name=0)

employees1

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7


In [62]:
employees2 = pd.read_excel('datasets/employee_information.xlsx', sheet_name='emergency_contacts', header=None, )

employees2.columns = ['employee_id', 'last_name', 'first_name', 'emergency_contact', 'emergency_contact_number', 'relationship']

employees2

,employee_id,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [63]:
employees3 = pd.read_json('datasets/employee_roles.json').T

employees3

,title,monthly_salary,team
A2R5H9,CEO,$4500,Leadership
H8K0L6,CFO,$4500,Leadership
G4R7V0,Business Developer,$3000,Sales
M1Z7U9,Office Manager,$2000,People Operations


In [64]:
# Merge all df into one
employees_combine = (employees1
 .merge(employees2, how='left', on='employee_id')
 .merge(employees3, how='left', left_on='employee_id', right_on=employees3.index)
 .merge(addresses, how='left', left_on='employee_country', right_on='office_country')
)

# Drop duplicate columns
employees_cleaned = (employees_combine
 .drop(['last_name', 'first_name'], axis=1)
 .set_index('employee_id')
)

In [65]:
# Renaming last_name, first_name column
employees_cleaned = (employees_cleaned.rename(
    {
        'employee_last_name': 'last_name',
        'employee_first_name': 'first_name'
    }, axis=1
))

In [66]:
# Fill missing values 
employees_cleaned['office'] = employees_cleaned['office'].fillna('Remote')

In [67]:
# Re-arange columns with .loc 
col_list = ['first_name', 'last_name', 'employee_country', 'employee_city', 'employee_street', 'employee_street_number', 'emergency_contact', 'emergency_contact_number', 'relationship', 'monthly_salary', 'team', 'title', 'office', 'office_country', 'office_city', 'office_street', 'office_street_number']

employees = employees_cleaned.loc[:, col_list]

In [68]:
employees

,first_name,last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,NaN,NaN,NaN,NaN
